In [2]:
#-------------------------------------------------------------------------
# QAOA More Complete Implementation
# Chapter 6 in the QUANTUM COMPUTING AND QUANTUM MACHINE LEARNING BOOK
#-------------------------------------------------------------------------
# Version 1.0
# (c) 2025 Jesse Van Griensven, Roydon Fraser, and Jose Rosas 
# Licence:  MIT - Citation required
#-------------------------------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt

from qiskit import Aer, QuantumCircuit, transpile
from qiskit.algorithms.optimizers import COBYLA
from qiskit.utils import QuantumInstance
from qiskit.opflow import PauliSumOp, StateFn, AerPauliExpectation, CircuitSampler
from qiskit.visualization import plot_histogram
#-------------------------------------------------------------------

#-------------------------------------------------------------------
def qaoa_circuit(gamma, beta):
# QAOA ansatz circuit (no classical register, no measurements)
    qc = QuantumCircuit(2)
    # Apply cost Hamiltonian (ZZ) as an RZZ rotation
    qc.rzz(2 * gamma, 0, 1)
    # Apply mixing Hamiltonian (global X rotations)
    qc.rx(2 * beta, 0)
    qc.rx(2 * beta, 1)
    return qc

#-------------------------------------------------------------------
def cost_function(params):
# Cost function:
#  1. Build an operator ~StateFn(H_C) @ StateFn(qaoa_circuit).
#  2. Convert it to a measurable circuit using CircuitSampler.
#  3. Evaluate the expectation value (return a float).

    gamma, beta = params
    # Build the QAOA ansatz circuit
    qc = qaoa_circuit(gamma, beta)
    
    # StateFn of the circuit
    ansatz_op = StateFn(qc)
    
    # We want <H_C> = <psi|H_C|psi>, so we create the operator:
    # ~StateFn(H_C) @ StateFn(psi)
    operator = ~StateFn(H_C) @ ansatz_op
    
    # Convert the operator to a circuit and evaluate using the sampler
    sampler = CircuitSampler(quantum_instance).convert(operator)
    value = sampler.eval().real  # Expectation value as a real number
    
    return value

#-------------------------------------------------------------------
def qaoa_circuit_with_measurements(gamma, beta):  
# Create a circuit with measurements
# to visualize results via histogram (counts).

    qc = qaoa_circuit(gamma, beta)
    qc.measure_all()
    return qc
#-------------------------------------------------------------------


# Define the cost Hamiltonian for a 2-qubit ZZ problem.
# For instance: H_C = ZZ
H_C = PauliSumOp.from_list([('ZZ', 1.0)])

# Create a QuantumInstance using the statevector simulator 
# (for cost function evaluation).
quantum_instance = QuantumInstance(backend=Aer.get_backend('statevector_simulator'))

# Optimize gamma and beta using COBYLA
optimizer = COBYLA()
initial_params = [0.5, 0.5]
result = optimizer.minimize(fun=cost_function, x0=initial_params)

# Print Results
print(f"Optimal parameters: gamma={result.x[0]:.4f}, beta={result.x[1]:.4f}")

# Create the measurement circuit using the optimal parameters
qc_meas = qaoa_circuit_with_measurements(result.x[0], result.x[1])

# Use the QASM simulator to generate counts (i.e., measure many shots)
backend = Aer.get_backend('qasm_simulator')
compiled_circuit = transpile(qc_meas, backend)
job = backend.run(compiled_circuit, shots=1024)
counts = job.result().get_counts()

# Plot histogram of measured states
plot_histogram(counts)
plt.show()


Optimal parameters: gamma=-0.4431, beta=-0.7853


/var/folders/z3/sk2c0q6j5jqfyd9r8pm2_vrr0000gp/T/ipykernel_52783/3506713452.py:71: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  quantum_instance = QuantumInstance(backend=Aer.get_backend('statevector_simulator'))
/var/folders/z3/sk2c0q6j5jqfyd9r8pm2_vrr0000gp/T/ipykernel_52783/3506713452.py:42: DeprecationWarning: The class ``qiskit.opflow.state_fns.circuit_state_fn.CircuitStateFn`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  ansatz_op = StateFn(qc)
/var/folders/z3/sk2c0q6j5jqfyd9r8pm2_vrr0000gp/T/ipykernel_52783/3506713452.py:46: DeprecationWarning: The class ``qiskit.opflow.state_fns.operator_state_fn.OperatorStateFn`` is deprecated as of 